In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import time

In [2]:
df = pd.read_pickle('./data/soccerDBNew2.pkl')

In [3]:
df['result'] = np.nan

In [4]:
for i in range(len(df)):
    if df.home_team_score[i] > df.away_team_score[i]:
        df['result'][i] = 'win'
    elif df.home_team_score[i] == df.away_team_score[i]:
        df['result'][i] = 'draw'
    else:
        df['result'][i] = 'lose'    

/home/renan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/renan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
def runModel(df, features):
    t0 = time.time()    
    data = df[['home_team', 'away_team', 'game', 'city', 'stadium']]
    label = df['result']
    data = pd.get_dummies(data)
    X_train, X_test, y_train, y_test = data[0:2735], data[2735:], label[0:2735], label[2735:]
    params = {
        'max_depth': 10,
        'objective': 'multi:softmax',  # error evaluation for multiclass training
        'num_class': 3,
        'n_gpus': 1
    }
    model = XGBClassifier(param=params)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    cm = confusion_matrix(y_test, pred)
    print ('run time: ', time.time()-t0,'s')
    print("Accuracy:", accuracy_score(y_test, pred))
    print (cm)

In [6]:
features = ['home_team', 'away_team', 'game', 'city', 'stadium', 'referee', 'datetime']

In [7]:
runModel(df,features)

run time:  2.978729486465454 s
Accuracy: 0.47514619883040937
[[  6   5 185]
 [  4   2 157]
 [  7   1 317]]
